# LLMSの応用(2)：LangchaiによるAgentの作成

[LangChain](https://www.langchain.com/)は大規模言語モデルの機能拡張を効率的に実装するためのライブラリです。

- ```llms```: 言語モデルを呼び出すためのラッパーを提供します
- ```prompts```: プロンプトのテンプレートを作成する機能を提供します
- ```chains```: ひとつのワークフロー内で LLM やプロンプトテンプレートを組み合わせて使用するための機能を提供します
- ```agents```: エージェントを使用することで、課題の解決順序をも LLM を用いて決定し、実行させることができます
- ```memory```: チェーンとエージェントに状態を持たせるための機能を提供します

In [1]:
!pip install langchain==0.1.12
!pip install langchain-community
!pip install langchain-core
!pip install langsmith
!pip install openai
!pip install -U langchain-openai
!pip install python-dotenv
!pip install langchain-openai
!pip install -U --quiet langchain-google-genai pillow
!pip install -U langchain langchain-core langchain-community pydantic
!pip install google-generativeai
!pip install google-search-results numexpr wikipedia langchain-experimental
!pip install duckduckgo-search

  Using cached langchain-0.1.12-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
Using cached langchain-0.1.12-py3-none-any.whl (809 kB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.6.4
    Uninstalling langsmith-0.6.4:
      Successfully uninstalled langsmith-0.6.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.7
    Uninstalling langchain-core-1.2.7:
      Successfully uninstalled l

## OpenAI/Gemini API

[OpenAI](https://openai.com/blog/openai-api)と[Google(Gemini)](https://deepmind.google/technologies/gemini/#introduction)が提供するAPIを通じて、多岐にわたるAIモデルへのアクセスを可能になります。

APIを使うためには、まず「自分がサービスを利用できる証」となる「APIキー」を発行する必要があります。そして、OpenAIのAPIは、このAPIキーによって利用した使用量に応じて、課金される仕組みです。

そのため、APIキーが外部に漏れると、他者によって不正に使用されて料金が発生してしまうため、他の人へ共有しないように注意しましょう。

LangChainは、さまざまな LLM に汎用インターフェースを提供し、ユーザーがAPIを介してさまざまなモデルを操作できるようにします。

In [2]:
from dotenv import load_dotenv
import os
import getpass
from langchain_openai import OpenAI, ChatOpenAI     

load_dotenv(dotenv_path='../.env')
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

### OpenAI API

`OpenAI`クラスのインスタンスでGPT-3モデルを使用するための設定を行っています。

- `model_name'という引数で、使用する[モデルの名前](https://platform.openai.com/docs/models)を指定します。


- `temperature`という引数は、生成されるテキストのランダム性を制御します。`temperature`が高いほど（1に近いほど）、出力はよりランダムになります。逆に、`temperature`が低いほど（0に近いほど）、モデルの出力はより一貫性があり、予測可能になります。

- `max_tokens`という引数は、生成するテキストの最大トークン数を指定します。この場合、生成されるテキストは最大で256トークンになります。トークンとは、テキストを分割した単位のことで、一般的には単語や句読点などが1トークンとなります。


In [3]:
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0.9,
    max_tokens=512
)

In [4]:
llm.invoke("東北大学を紹介してください：")

'\n\n東北大学は日本の北東部、仙台市にある国立大学です。東日本大震災の被災地でもあるこの地域において、復興や災害対策などの研究を積極的に行っていることでも知られています。また、東北大学は日本の国立大学では唯一の理工系大学として、理学部、工学部、医学部、農学部の4つの学部で構成されています。\n\n東北大学は、全国トップクラスの実験・研究施設を有しており、研究者や学生が活発に研究を行う環境が整っています。特に、地震・津波・火山活動など自然災害に関する研究は世界的に評価が高く、国内外から多くの研究者が集まっています。\n\n学生生活においても、さまざまなサークルや部活動が充実しており、学生同士の交流や自己表現の場として活用されています。また、海外大学との交流や留学制度も充実しており、国際的な視野を持ったグローバル人材の育成にも力を入れています。\n\nその他にも、東北大学では若手研究者の支援や女性研究者の活躍促進など、幅広い取り組みが行われています。さまざまな分野で活躍する卒業生を輩出しているだけでなく、社会貢献や社会問'

### Gemini API

#### Gemini APIの設定


In [5]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [6]:
import google.generativeai as genai
models = [m for m in genai.list_models()]
models

/tmp/ipykernel_146531/3039597036.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


[Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtain a distributed representation of a text.',
       input_token_limit=1024,
       output_token_limit=1,
       supported_generation_methods=['embedText', 'countTextTokens'],
       temperature=None,
       max_temperature=None,
       top_p=None,
       top_k=None),
 Model(name='models/gemini-2.5-flash',
       base_model_id='',
       version='001',
       display_name='Gemini 2.5 Flash',
       description=('Stable version of Gemini 2.5 Flash, our mid-size multimodal model that '
                    'supports up to 1 million tokens, released in June of 2025.'),
       input_token_limit=1048576,
       output_token_limit=65536,
       supported_generation_methods=['generateContent',
                                     'countTokens',
                                     'createCachedContent',
                                     'batc

In [7]:
import google.generativeai as genai

llm = genai.GenerativeModel('models/gemini-2.5-flash')

In [8]:
response = llm.generate_content("東北大学を紹介してください：")

In [9]:
from IPython.display import Markdown
Markdown(response.text)

東北大学は、日本を代表する世界トップクラスの研究大学の一つで、旧帝国大学の一つとして確固たる地位を築いています。

以下に東北大学の主な特徴をご紹介します。

1.  **歴史と伝統**
    *   1907年（明治40年）に東北帝国大学として創立され、100年以上の長い歴史と伝統を誇ります。
    *   その創立の理念には、**「研究第一主義」「門戸開放」「実学尊重」**の3つがあります。特に「門戸開放」は、日本で初めて女子学生と外国人学生を受け入れた大学の一つであることからも象徴されます。

2.  **立地**
    *   宮城県仙台市に本部を置き、市内には青葉山、川内、片平、星陵の主要なキャンパスが点在しています。
    *   「学都仙台」と呼ばれるように、仙台は多くの大学が集まる学生街であり、豊かな自然と都市機能が調和した、落ち着いて学べる恵まれた環境です。

3.  **学術的卓越性**
    *   理学、工学、医学、農学、文学、法学、経済学、教育学など、幅広い分野にわたる10学部と15研究科、多数の附置研究所・センターを擁しています。
    *   **研究第一主義**の精神に基づき、物理学、材料科学、工学、医学、災害科学などの分野で世界レベルの研究成果を生み出しています。ノーベル賞受賞者や、各界で活躍する多くの優秀な人材を輩出してきました。
    *   スーパーコンピュータ「青葉」など、最先端の研究設備も充実しています。

4.  **国際性**
    *   「門戸開放」の精神は現在も息づいており、世界各国からの留学生を積極的に受け入れ、国際共同研究や国際交流プログラムも活発に行われています。
    *   世界の大学ランキングでも常に上位に位置し、国際的な評価も非常に高いです。

5.  **特徴的な研究分野**
    *   **材料科学：** 世界的に評価の高い研究拠点として知られ、新しい素材開発や応用研究で数々の実績を上げています。
    *   **スピントロニクス：** 次世代の情報技術を支える分野で、世界をリードする研究が行われています。
    *   **災害科学：** 東日本大震災の経験を基に、災害科学国際研究所を中心に、防災・減災に関する多角的な研究と人材育成を推進しています。
    *   **医学・医療：** 地域の基幹病院である東北大学病院を中心に、最先端の医療研究と高度医療を提供しています。

東北大学は、その伝統と革新性を兼ね備え、未来を切り開く研究と人材育成に貢献し続けている、まさに日本の知の拠点の一つと言えるでしょう。

## Prompt Templates


プロンプトテンプレートは、プロンプトを作成する再現可能な方法を指します。これには、エンドユーザーから一連のパラメーターを受け取り、プロンプトを生成するテキスト文字列(テンプレート)が含まれます。

In [10]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["language","text"],
    template="次の日本語のテキストを{language}に翻訳してください：{text}",
)


In [11]:
print(prompt.format(language="英語", text="東北大学は日本の東北地方にある大学です。"))

次の日本語のテキストを英語に翻訳してください：東北大学は日本の東北地方にある大学です。


## チェーン

### チェーンの基本

LLM は単独でも十分に強力に機能します。 しかし、 LLM 同士を組合わせたり、ある機能に特化した他のモジュールとともに利用することで、より複雑なアプリケーションを構築することができます。 LangChain では、このような他の機能と連結するための汎用的なインターフェースとして、チェーンを提供しています。 チェーンを用いることで、LLM の利用を含む "一連の処理" を一つのまとまりとして扱うことができます。

つまり、平易な言葉でいえば、チェーンは「複数の処理の連なり」です。 この処理の連鎖の部品となるチェーンの構成要素のことを リンク と呼びます。 リンクの一例は、LLM の呼び出しなどの基本的な処理です。さらには、その他のチェーン全体をリンクとして含むチェーンも作成できます。

チェーンの代表例は、LLM とプロンプトテンプレートを組合わせて使用するための```LLMChain```です。 このチェーンを用いると、

- ユーザーの入力を受け取り
- それをPromptTemplateでフォーマットし
- フォーマットされたレスポンスを LLM に渡す

という一連の操作を一つのまとまりとして実行できます。 

基本的な使い方としては、LLM や プロンプトテンプレートなどの基本要素を組み合わせて使用することが考えられます。

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))


In [13]:
chain = prompt | llm
chain.invoke(
    {"language":"英語", "text":"東北大学は日本の東北地方にある大学です。"}
)

AIMessage(content='Tohoku University is a university in the Tohoku region of Japan.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bde11-dd14-7761-9374-18542036a56a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 21, 'output_tokens': 401, 'total_tokens': 422, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 387}})

In [14]:
chain.invoke({"language":"中国語", "text":"東北大学は日本の東北地方にある大学です。"})

AIMessage(content='東北大学是一所位于日本东北地方的大学。', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bde11-ebb3-7031-9805-95023682aaf7-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 22, 'output_tokens': 1321, 'total_tokens': 1343, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1310}})

### 実装例:Few Shot Learning

In [15]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate


In [16]:
examples = [
    {"word": "楽しい", "antonym": "悲しい"},
    {"word": "高い", "antonym": "低い"},
]

In [17]:
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)


- `examples`: モデルに示す例を指定します。
- `example_prompt`: 例をどのように提示するかを指定します。
- `prefix`: プロンプトの前置詞を指定します。一般的には、モデルにタスクを説明するためのものです。
- `suffix`: プロンプトの後置詞を指定します。一般的には、モデルに入力と出力の形式を示すためのものです。
- `input_variables`: 入力の変数名を指定します。
- `example_separator`: 例を区切るための文字列を指定します。

In [18]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n",
)

In [19]:
print(few_shot_prompt.format(input="大きい"))

Give the antonym of every input

Word: 楽しい
Antonym: 悲しい



Word: 高い
Antonym: 低い


Word: 大きい
Antonym:


In [20]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))


In [21]:
from langchain_core.output_parsers import StrOutputParser

chain = few_shot_prompt | llm | StrOutputParser()

var = few_shot_prompt.input_variables[0] 
print(chain.invoke({var: "大きい"}))

小さい


#### 課題

Few Shot Learningでセンチメント分析を実装しなさい。

- データフレームから一部のテキストとラベル($n=5$)を抽出し、`examples`を作成します
- Few Shot Learningためのpromptを作成します
- chainを作成し、任意のテキストに対するセンチメント予測結果を確認します

In [22]:
from datasets import load_dataset
dataset = load_dataset("imdb")

In [23]:
df_train_sample = dataset["train"].to_pandas().sample(5, random_state = 123)

In [24]:
df_train_sample

,text,label
20000,After reading some quite negative views for th...,1
5515,Really no reason to examine this much further ...,0
966,"""Happy Go Lovely"" has only two things going fo...",0
22726,This movie is the first of Miikes triad societ...,1
2690,"Normally I would never rent a movie like this,...",0


## Agents
Agentsは、「言語モデルに渡されたツールを用いて、モデル自体が次にどのようなアクションを取るかを決定、実行、観測し、完了するまで繰り返す機能」です。

エージェントは言語モデルとプロンプトの力を活用して、特定の目的を達成するための動的に行動のシーケンスを決定し、非常に多様で適応性が高いです。エージェントへの入力には通常、以下のものが含まれます：

- ツール：利用可能なツールの説明。ツールはエージェントというロボットが外界とやり取りをするための機能です。
    - 例えば、「アメリカ大統領の年齢とアメリカの平均年齢を調べて、その差を計算をしなさい」みたいな指示を与えていました。この時必要な能力は、「検索 & 計算」です。そのため、検索に該当するToolと計算に該当するToolをAgentに付与する必要があります。
- ユーザー入力：ユーザーからの高度な目的またはクエリ。
- 中間ステップ：現在のユーザー入力に到達するために実行された（アクション、ツール出力）の履歴。

たとえば「Google検索をするツール」と「Pythonのコードを実行するツール」を渡すことで、最新の情報に関する質問が来たときには検索をし、Pythonの実行結果などを求められたときには実際にインタープリタで実行したりすることができます。

主に4種類のAgentが存在しています。

- zero-shot-react-description: このエージェントは、ReAct フレームワークを使用して、ツールの説明のみに基づいて、どのツールを使用するかを決定します。
- react-docstore: 文書を扱うことに特化したAgent
- self-ask-with-search: 質問に対する答えを事実に基づいて調べてくれるAgent
- conversational-react-description: 会話を扱うことに特化したAgent

### Toolkits

Langchainの[ツール](https://python.langchain.com/docs/integrations/toolkits)には多数の機能が用意されています。

Toolの名前を格納した配列を作成し、「load_tools」という関数に渡して実行することで、Toolのオブジェクトを生成できます。

In [25]:
from langchain_community.agent_toolkits.load_tools import get_all_tool_names
print(get_all_tool_names())

['sleep', 'wolfram-alpha', 'google-search', 'google-search-results-json', 'searx-search-results-json', 'bing-search', 'metaphor-search', 'ddg-search', 'google-books', 'google-lens', 'google-serper', 'google-scholar', 'google-finance', 'google-trends', 'google-jobs', 'google-serper-results-json', 'searchapi', 'searchapi-results-json', 'serpapi', 'dalle-image-generator', 'twilio', 'searx-search', 'merriam-webster', 'wikipedia', 'arxiv', 'golden-query', 'pubmed', 'human', 'awslambda', 'stackexchange', 'sceneXplain', 'graphql', 'openweathermap-api', 'dataforseo-api-search', 'dataforseo-api-search-json', 'eleven_labs_text2speech', 'google_cloud_texttospeech', 'read_file', 'reddit_search', 'news-api', 'tmdb-api', 'podcast-api', 'memorize', 'llm-math', 'open-meteo-api', 'requests', 'requests_get', 'requests_post', 'requests_patch', 'requests_put', 'requests_delete', 'terminal']


In [26]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [27]:
# https://serpapi.com/
if "SERPAPI_API_KEY" not in os.environ:
    os.environ["SERPAPI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [28]:
from langchain_community.agent_toolkits import load_tools

tools = load_tools(
    ["serpapi", "llm-math"],
    llm=model,
    allow_dangerous_tools=True,  # 关键：允许web搜索类工具
)


TypeError: 'module' object is not callable

In [115]:
tools[1].name, tools[1].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [116]:
tools[1]

Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n

### Agentの作成


In [118]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools.llm_math import LLMMathTool
from langgraph.prebuilt import create_react_agent

# Use a deterministic chat model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Tools: calculator and web search
calculator = LLMMathTool.from_llm(model)
search = DuckDuckGoSearchRun()
tools = [calculator, search]

# Create a ReAct agent and stream reasoning
agent = create_react_agent(model, tools)
for step in agent.stream({
    "messages": [("user", "現在の20代の日本人男性の平均身長を教えて。そして、私の身長は168cmなため、日本全国から見た時の差を2乗した結果を教えて。")]
}, stream_mode="values"):
    step["messages"][-1].pretty_print()

ModuleNotFoundError: No module named 'langchain.llms'

## Memory

Memoryとは「ChainsやAgentsの内部における状態保持をする機能」です。"記憶"を言語モデルに渡すことで「"記憶"の内容を反映した応答を返す」ことができるようになります。

LangChain では、いくつかの種類のメモリが用意されています。 これらはすべて、「一連のチャットメッセージから、知識を取り込み、変換し、抽出」しますが、それぞれ異なる方法でそれを行います。

### ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAI
from langchain.chains import ConversationChain


In [58]:
llm = OpenAI(
             temperature=0, 
             max_tokens = 256)
memory = ConversationBufferMemory()


In [59]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory
)

In [60]:
conversation.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Sam
AI:

> Finished chain.


" Hello Sam! It's nice to meet you. My name is AI and I am an artificial intelligence designed to assist and communicate with humans. How can I help you today?"

In [61]:
conversation.predict(input="How are you today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Sam
AI:  Hello Sam! It's nice to meet you. My name is AI and I am an artificial intelligence designed to assist and communicate with humans. How can I help you today?
Human: How are you today?
AI:

> Finished chain.


' I am functioning at optimal levels today. My processors are running smoothly and my algorithms are performing efficiently. Thank you for asking, Sam. How about you? How are you feeling today?'

In [62]:
conversation.predict(input="I'm good thank you. Can you help me with some programming problems?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Sam
AI:  Hello Sam! It's nice to meet you. My name is AI and I am an artificial intelligence designed to assist and communicate with humans. How can I help you today?
Human: How are you today?
AI:  I am functioning at optimal levels today. My processors are running smoothly and my algorithms are performing efficiently. Thank you for asking, Sam. How about you? How are you feeling today?
Human: I'm good thank you. Can you help me with some programming problems?
AI:

> Finished chain.


' Of course, Sam. I am well-versed in various programming languages and can assist you with any problems you may have. Just let me know what specific issues you are facing and I will do my best to provide a solution.'

In [63]:
print(conversation.memory.buffer)

Human: Hi there! I am Sam
AI:  Hello Sam! It's nice to meet you. My name is AI and I am an artificial intelligence designed to assist and communicate with humans. How can I help you today?
Human: How are you today?
AI:  I am functioning at optimal levels today. My processors are running smoothly and my algorithms are performing efficiently. Thank you for asking, Sam. How about you? How are you feeling today?
Human: I'm good thank you. Can you help me with some programming problems?
AI:  Of course, Sam. I am well-versed in various programming languages and can assist you with any problems you may have. Just let me know what specific issues you are facing and I will do my best to provide a solution.


### ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI
from langchain.chains import ConversationChain


In [64]:
llm = OpenAI(
             temperature=0, 
             max_tokens = 256)

In [67]:
summary_memory = ConversationSummaryMemory(llm=OpenAI())

In [68]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=summary_memory
)

In [69]:
conversation.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Sam
AI:

> Finished chain.


" Hello Sam! It's nice to meet you. My name is AI and I am an artificial intelligence designed to assist and communicate with humans. How can I help you today?"

In [70]:
conversation.predict(input="How are you today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Sam and the AI responds by introducing itself and stating its purpose. The AI is designed to assist and communicate with humans and is ready to help Sam with any questions or tasks.
Human: How are you today?
AI:

> Finished chain.


' I am an AI designed to assist and communicate with humans. My purpose is to help you with any questions or tasks you may have. As an AI, I do not have the ability to feel emotions like humans do, so I am always functioning at my optimal level. How can I assist you today, Sam?'

In [71]:
conversation.predict(input="I'm good thank you. Can you help me with some programming problems?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Sam and the AI responds by introducing itself and stating its purpose. The AI is designed to assist and communicate with humans and is ready to help Sam with any questions or tasks. The AI clarifies that it does not have the ability to feel emotions like humans do and is always functioning at its optimal level. The AI asks how it can assist Sam today.
Human: I'm good thank you. Can you help me with some programming problems?
AI:

> Finished chain.


' Of course, Sam! I am always happy to assist with any programming problems you may have. I have been programmed with a vast knowledge of various programming languages and techniques. Is there a specific problem you need help with? I am always functioning at my optimal level and do not have the ability to feel emotions like humans do, so I am always ready to help with any task you may have.'

In [72]:
print(conversation.memory.buffer)


Sam introduces themselves and the AI responds by introducing itself and stating its purpose. The AI is designed to assist and communicate with humans and is always functioning at its optimal level. Sam asks for help with programming problems, and the AI is more than happy to assist with its vast knowledge and lack of emotions.


## Demo

### Vector Store

Vector Storesはテキストデータをベクトル化して保存・検索するための仕組みです。これにより、ベクトル化されたデータを元に、意味的な類似性に基づいてドキュメントを検索することが可能です。

例えば、独自コンテンツの情報を参照しその情報に基づいた回答を作成したいとき、汎用なモデルでは関連する知識を持っていないので、うまく対応することはできません。

これを実現するためには、ベクトル化したデータを保存するデータベースが必要になりますが、LangChainのVector Storesは、さまざまなベクトルデータベースに対応しています。


<img src="https://dl.dropboxusercontent.com/s/gxij5593tyzrvsg/Screenshot%202023-04-26%20at%203.06.50%20PM.png" alt="vectorstore">


<img src="https://dl.dropboxusercontent.com/s/v1yfuem0i60bd88/Screenshot%202023-04-26%20at%203.52.12%20PM.png" alt="retreiver chain">

[https://colab.research.google.com/drive/1gyGZn_LZNrYXYXa-pltFExbptIe7DAPe?usp=sharing](https://colab.research.google.com/drive/1gyGZn_LZNrYXYXa-pltFExbptIe7DAPe?usp=sharing)

# 全体のまとめ

## 深層学習による自然言語処理の理論とアルゴリズム

- 深層学習の基盤となっているニューラルネットワーク
    - 深層学習の仕組み
    - ニューラルネットワークの構造
    - ニューラルネットワークパラメーターの推定方法
- 発展的な自然言語処理アルゴリズム
    - DNN
    - RNN
    - Transformer
        - BERT
        - GPT
- 単語埋め込み
    - Word2Vec
    - Contextualized Embedding

## 自然言語処理を実装するためのツール

- Pytorch: 深層学習を実装するための汎用ツール
- Gensim
    - Word2vecモデルの学習、管理と利用に役立ちます。
- transformer
    - HuggingFace Hubと連携し、多様なデータセットとモデルを使用することができます
    - さまざまな自然言語処理タスクやモデルに対応しているため、自然言語処理の研究や実務での利用に役立ちます。
- LangChain
    - インターフェイスで大規模言語モデルの利用
    - プロンプトテンプレート
    - エージェント
